In [20]:
import torch
import pandas as pd
from pathlib import Path
from transformers import pipeline
import torch.nn as nn
import numpy as np

In [3]:
def run(args):
    print(args)
    # Seeds for reproducibility (By default we use the number 21)

    # Load sales data
    data_folder = "./dataset/"

    # Load category and color encodings
    cat_dict = torch.load(Path(data_folder + 'category_labels.pt'))
    col_dict = torch.load(Path(data_folder + 'color_labels.pt'))
    fab_dict = torch.load(Path(data_folder + 'fabric_labels.pt'))

    print(cat_dict)
    print(col_dict)
    print(fab_dict)

    return cat_dict, col_dict, fab_dict

In [4]:
cat_dict, col_dict, fab_dict = run(5)

5
{'capris': 0, 'cloak': 1, 'culottes': 2, 'doll dress': 3, 'drop sleeve': 4, 'gitana skirt': 5, 'kimono dress': 6, 'long cardigan': 7, 'long coat': 8, 'long dress': 9, 'long duster': 10, 'long skirt': 11, 'long sleeve': 12, 'longuette skirt': 13, 'maxi': 14, 'medium cardigan': 15, 'medium coat': 16, 'medium duster': 17, 'midi skirt': 18, 'miniskirt': 19, 'patterned': 20, 'printed': 21, 'sheath dress': 22, 'shirt dress': 23, 'short cardigan': 24, 'short coat': 25, 'short sleeves': 26, 'shorts': 27, 'sleeveless': 28, 'solid colours': 29, 'tracksuit': 30, 'trapeze dress': 31}
{'yellow': 0, 'brown': 1, 'blue': 2, 'grey': 3, 'green': 4, 'black': 5, 'red': 6, 'white': 7, 'orange': 8, 'violet': 9}
{'acrylic': 0, 'scuba crepe': 49, 'tulle': 55, 'angora': 1, 'faux leather': 14, 'georgette': 21, 'lurex': 30, 'nice': 38, 'crepe': 9, 'satin cotton': 47, 'silky satin': 51, 'fur': 20, 'matte jersey': 33, 'plisse': 43, 'velvet': 56, 'lace': 27, 'cotton': 8, 'piquet': 42, 'plush': 45, 'bengaline': 2,

In [5]:
class TextEmbedder(nn.Module):
    def __init__(self, embedding_dim, cat_dict, col_dict, fab_dict, gpu_num):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.cat_dict = {v: k for k, v in cat_dict.items()}
        self.col_dict = {v: k for k, v in col_dict.items()}
        self.fab_dict = {v: k for k, v in fab_dict.items()}
        self.word_embedder = pipeline('feature-extraction', model='bert-base-uncased')
        self.fc = nn.Linear(768, embedding_dim)
        self.dropout = nn.Dropout(0.1)
        self.gpu_num = gpu_num

    def forward(self, category, color, fabric):
        textual_description = [self.col_dict[color.detach().cpu().numpy().tolist()[i]] + ' ' \
                + self.fab_dict[fabric.detach().cpu().numpy().tolist()[i]] + ' ' \
                + self.cat_dict[category.detach().cpu().numpy().tolist()[i]] for i in range(len(category))]
        # Hafta ve mağaza idsi eklenmeli.


        # Use BERT to extract features
        word_embeddings = self.word_embedder(textual_description)

        # BERT gives us embeddings for [CLS] ..  [EOS], which is why we only average the embeddings in the range [1:-1] 
        # We're not fine tuning BERT and we don't want the noise coming from [CLS] or [EOS]
        word_embeddings = [torch.FloatTensor(x[0][1:-1]).mean(axis=0) for x in word_embeddings] 
        word_embeddings = torch.stack(word_embeddings)
        
        # Embed to our embedding space
        word_embeddings = self.dropout(self.fc(word_embeddings))

        return word_embeddings

In [6]:
text_encoder = TextEmbedder(32, cat_dict, col_dict, fab_dict, 0)

In [25]:
from utils.data_multitrends import ZeroShotDataset
import pandas as pd
from pathlib import Path

In [22]:
train_df = pd.read_csv(Path("./dataset/" + 'train.csv'), parse_dates=['release_date'])
test_df = pd.read_csv(Path("./dataset/" + 'test.csv'), parse_dates=['release_date'])

gtrends = pd.read_csv(Path("./dataset/" + 'gtrends.csv'), index_col=[0], parse_dates=True)

train_loader = ZeroShotDataset(train_df, Path("./dataset/" + '/images'), gtrends, cat_dict, col_dict,
                                fab_dict, 52).get_loader(batch_size=128, train=True)
test_loader = ZeroShotDataset(test_df, Path("./dataset/" + '/images'), gtrends, cat_dict, col_dict,
                                fab_dict, 52).get_loader(batch_size=1, train=False)

Starting dataset creation process...


 23%|##3       | 1187/5080 [00:14<00:46, 82.86it/s]


KeyboardInterrupt: 

### New

In [27]:
train_df = pd.read_csv(Path("./dataset/" + 'train.csv'), parse_dates=['release_date'])
test_df = pd.read_csv(Path("./dataset/" + 'test.csv'), parse_dates=['release_date'])

gtrends = pd.read_csv(Path("./dataset/" + 'gtrends.csv'), index_col=[0], parse_dates=True)

train_loader = ZeroShotDataset(data_df=train_df, gtrends=gtrends, cat_dict=cat_dict, col_dict=col_dict,
                                fab_dict=fab_dict, trend_len=52).get_loader(batch_size=128, train=True)
test_loader = ZeroShotDataset(test_df, gtrends, cat_dict, col_dict,
                                fab_dict, 52).get_loader(batch_size=1, train=False)

TypeError: __init__() missing 1 required positional argument: 'img_root'

In [10]:
# Epoch
for data in train_loader:
    item_sales, category, color, fabric, temporal_features, gtrends, images = data 
    break
    

In [11]:
text_encoding = text_encoder(category, color, fabric)

In [12]:
text_encoding

tensor([[-0.1781,  0.0698, -0.4261,  ..., -0.0253, -0.3746, -0.0020],
        [-0.2398,  0.0000, -0.4868,  ..., -0.4519,  0.0000, -0.1146],
        [ 0.0041,  0.2440, -0.6640,  ..., -0.4740,  0.2231,  0.2407],
        ...,
        [-0.2707,  0.1398, -0.5094,  ..., -0.0000, -0.0000,  0.2750],
        [-0.1369,  0.2979, -0.2432,  ...,  0.0048,  0.1349, -0.1145],
        [-0.0000,  0.0000, -0.0000,  ..., -0.1628,  0.0000,  0.0815]],
       grad_fn=<MulBackward0>)

In [13]:
len(text_encoding)

128

In [14]:
len(text_encoding[0])

32